In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install all the requirements
!pip install langchain-groq
!pip install chromadb
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 30.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.1/602.1 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    groq_api_key = '',
    model_name = "llama-3.3-70b-versatile",
    temperature=0
)

#LLM Model Response Test
response = llm.invoke("Hey there, which model are you")
print(response.content)

I'm an AI designed by Meta AI, and I'm a type of large language model. My knowledge was built from a massive corpus of text data, and I'm constantly learning and improving my responses. I don't have a personal name, but I'm often referred to as a "LLaMA" model, which stands for "Large Language Model Meta AI."


In [ ]:
OPEN_API_KEY = ""

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader

# Load the CSV data for Langchain
loader = CSVLoader(file_path="/content/drive/My Drive/train.csv")
data = loader.load()
# Print the number of documents
print(len(data))

3512


In [ ]:
print(type(data))

<class 'list'>


In [ ]:
#Iterate and store the output data in a sperate list as dictionaries
output_data = []
# Loop through each document in the loaded data
for doc in data:
    # Get the page content (which contains both context and response)
    page_content = doc.page_content

    # Split the page content into context and response
    if "Response:" in page_content:
        context = page_content.split("Response:")[0].replace("Context:", "").strip()
        response = page_content.split("Response:")[1].strip()

        # Append the extracted context and response to the output_data list
        output_data.append({
            "context": context,
            "response": response
        })
    else:
        print("Response not found in page content.")

In [ ]:
import pandas as pd
df = pd.DataFrame(output_data)
print(df.head())

                                             context  \
0  I'm going through some things with my feelings...   
1  I'm going through some things with my feelings...   
2  I'm going through some things with my feelings...   
3  I'm going through some things with my feelings...   
4  I'm going through some things with my feelings...   

                                            response  
0  If everyone thinks you're worthless, then mayb...  
1  Hello, and thank you for your question and see...  
2  First thing I'd suggest is getting the sleep y...  
3  Therapy is essential for those that are feelin...  
4  I first want to let you know that you are not ...  


In [ ]:
!pip install tiktoken
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.9 MB/s eta 0:00:00


In [ ]:
#Uploading the Dataset as collections into chromadb if it's not present already
import uuid
import chromadb
from langchain.embeddings import OpenAIEmbeddings


embedding = OpenAIEmbeddings(openai_api_key=OPEN_API_KEY, model='text-embedding-3-small')



client = chromadb.PersistentClient(path="/content/drive/My Drive/chromadbnew")
collection = client.get_or_create_collection("queries")

<ipython-input-10-9b3c062dbb5f>:7: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(openai_api_key=OPEN_API_KEY, model='text-embedding-3-small')


In [ ]:
import uuid
# Add embeddings to the vector database
if not collection.count():
    for _, row in df.iterrows():
        try:
            # Generate embedding
            context_embedding = embedding.embed_documents([row["context"]])[0]
            collection.add(
                documents=[row["context"]],
                embeddings=[context_embedding],
                metadatas=[{"response": row["response"]}],
                ids=[str(uuid.uuid4())]
            )
        except Exception as e:
            print(f"Error adding row: {row}, Error: {e}")

In [ ]:
# Input question
question = "give me some motivation"

# Generate embedding for the question
question_embedding = embedding.embed_documents([question])[0]

# Query the vector database
results = collection.query(
    # Use the embedding of the question
    query_embeddings=[question_embedding],
    # Retrieve top 5 responses
    n_results=5
)

for idx, metadata_list in enumerate(results["metadatas"]):
    # Each "metadata_list" is a list; access the first dictionary in it
    if metadata_list:
        print(f"Result {idx+1}: {metadata_list[0]['response']}")
        print(len(results))
        print(type(results))
        print(results.keys())
        print(results['documents'])
        print(results['metadatas'])
        print(results)

Result 1: Anxiety and Depression are challenging experiences to live with and to manage on a daily basis. I would say that both are challenges to overcome but solutions to living healthy and well exist. Step 1: Talk about it. With friends, family, partners, counselors, and other trusted people in your life. Step 2: Create a plan with a counselor to learn new skills that help you recognize and manage your symptoms. Step 3: Don't give up. Working on yourself can be difficult and hard at the beginning. Stick with it and you will be able to find exercises, tools, and resources that help you live well.
8
<class 'dict'>
dict_keys(['ids', 'embeddings', 'documents', 'uris', 'data', 'metadatas', 'distances', 'included'])
[["My motivation has gone away. It's hard to get out of bed. I really don't know what to do anymore. I'm miserable. My anxiety and depression have taken over my life.", "My motivation has gone away. It's hard to get out of bed. I really don't know what to do anymore. I'm misera

In [ ]:
from langchain.prompts import PromptTemplate



#Building the Prompt Template
prompt_extract = PromptTemplate.from_template(
    """
    ### This is a data from mental health counselling dataset
    {retrieved_responses}
    ### Instruction:
    You are a mental health therapist
    This data represents a collection of mental health counselling records, including both the context (question) and the response.
    Use this data context and response to provide a summary answer to the question asked.
    Keep the answer short
    If the question is irrelevant just tell it's an irrelevant context
    ### Question:
    {question}
    ### Response:
    """
)

#Chain Extract
chain_extract = prompt_extract | llm

In [ ]:
#Question
# Input question
question = "give me some motivation"

# Generate embedding for the question
question_embedding = embedding.embed_documents([question])[0]

# Query the vector database
results = collection.query(
    # Use the embedding of the question
    query_embeddings=[question_embedding],
    # Retrieve top 5 responses
    n_results=5
)
# Combine responses into a single string
retrieved_responses = "\n".join(
    [metadata["response"] for metadata_list in results["metadatas"] for metadata in metadata_list]
)

res = chain_extract.invoke(input={"retrieved_responses": retrieved_responses, "question": question})

#Printed Response
print(f"Response: \n{res.content}")

Response: 
To find motivation, start by establishing a good self-care routine: eat balanced meals, sleep 6-8 hours, and exercise 30 minutes a day. This helps regulate your mood. If struggling, talk to a therapist to identify motivations and consider discussing medication options with a physician or psychiatrist. Remember, small steps with self-compassion can help you build a stronger foundation to overcome anxiety and depression.
